# UPPP 214 - Week 7: Z & T tests

<a target="_blank" href="https://colab.research.google.com/github/knaaptime/uppp214-winter26-assn/blob/main/week7/z-t-tests.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
import numpy as np
import pandas as pd
import scipy

In [2]:
df = pd.read_parquet(
    "https://github.com/oturns/example_datasets/raw/refs/heads/main/acs/ca_tracts_2021.pq"
)

In [3]:
df

,geoid,n_mexican_pop,n_cuban_pop,n_puerto_rican_pop,n_russian_pop,n_italian_pop,n_german_pop,n_irish_pop,n_scandaniavian_pop,n_foreign_born_pop,...,p_poverty_rate,p_poverty_rate_over_65,p_poverty_rate_children,p_poverty_rate_white,p_poverty_rate_black,p_poverty_rate_hispanic,p_poverty_rate_native,p_poverty_rate_asian,geometry,year
0,06001400100,177.0,0.0,0.0,40.0,27.0,91.0,23.0,0.0,894.0,...,5.221853,1.961968,0.000000,2.867492,0.392394,2.626019,0.0,0.000000,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...,2021
1,06001400200,78.0,0.0,4.0,8.0,14.0,34.0,38.0,0.0,275.0,...,6.575074,1.226693,0.000000,5.152110,0.392542,1.570167,0.0,0.000000,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...,2021
2,06001400300,225.0,19.0,18.0,34.0,245.0,39.0,69.0,0.0,843.0,...,4.879852,1.182994,0.000000,2.439926,1.626617,0.536044,0.0,0.000000,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...,2021
3,06001400400,302.0,0.0,27.0,9.0,88.0,21.0,76.0,0.0,433.0,...,8.827555,1.254562,1.094891,3.170620,1.984489,3.786496,0.0,0.000000,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...,2021
4,06001400500,242.0,47.0,19.0,27.0,31.0,27.0,51.0,0.0,804.0,...,11.141372,1.213172,0.198069,4.926962,3.020550,0.693241,0.0,0.000000,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9124,06115040902,195.0,0.0,12.0,0.0,0.0,89.0,26.0,0.0,71.0,...,9.990394,0.000000,2.401537,5.763689,0.000000,2.017291,0.0,0.000000,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...,2021
9125,06115041001,772.0,0.0,184.0,0.0,55.0,135.0,50.0,0.0,379.0,...,8.997693,1.614971,0.333248,5.280697,0.000000,1.153550,0.0,0.000000,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...,2021
9126,06115041002,251.0,13.0,0.0,0.0,0.0,133.0,101.0,0.0,462.0,...,5.434141,0.000000,0.000000,5.138807,0.000000,1.919669,0.0,0.000000,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...,2021
9127,06115041101,215.0,0.0,0.0,0.0,6.0,30.0,32.0,0.0,135.0,...,18.702290,1.946565,1.145038,12.824427,2.328244,3.511450,0.0,0.152672,b'\x01\x06\x00\x00\x00\x01\x00\x00\x00\x01\x03...,2021


In [4]:
la = df[df.geoid.str.startswith("06037")]

oc = df[df.geoid.str.startswith("06059")]

In [5]:
la.shape

(2498, 158)

In [6]:
oc.shape

(614, 158)

## t-test by hand

we're curious about the relationship between counties and home values. 

Do LA and OC have the same housing market?

How do we write this as a formal hypothesis?


In [7]:
la.median_home_value.mean()

np.float64(713659.7960750853)

In [8]:
oc.median_home_value.mean()

np.float64(780870.5333333333)

In [9]:
oc.median_home_value.mean() - la.median_home_value.mean()

np.float64(67210.73725824803)

$H_0$: OC is less than or equal to LA


$H_A$: OC is more expensive than LA

In [ ]:
oc_mean = oc.median_home_value.dropna().mean()
oc_n = oc.dropna(subset="median_home_value").shape[0]

la_mean = la.median_home_value.dropna().mean()
la_n = la.dropna(subset="median_home_value").shape[0]

now we need the [pooled standard deviation](https://ethanweed.github.io/pythonbook/05.02-ttest.html#a-pooled-estimate-of-the-standard-deviation)

$$ \hat\sigma_p = \sqrt{\frac{w_1 {\hat\sigma_1}^2 + w_2 {\hat\sigma_2}^2}{w_1 + w_2}} $$

In [ ]:
sp = np.sqrt(
    (
        ((oc_n - 1) * oc.median_home_value.dropna().std() ** 2)
        + ((la_n - 1) * la.median_home_value.dropna().std() ** 2)
    )
    / (la_n + oc_n - 2)
)

In [ ]:
sp

In [ ]:
(oc_mean - la_mean)

In [ ]:
np.sqrt((1 / oc_n) + (1 / la_n))

In [ ]:
sp * np.sqrt((1 / (oc_n) + (1 / la_n)))

In [ ]:
t = (oc_mean - la_mean) / (sp * np.sqrt(((1 / oc_n) + (1 / la_n))))

In [ ]:
t

to calculate the p-value, we need the T-statistic and degrees of freedom

degrees of freedom  = n1 + n2 -2

In [ ]:
dof = oc_n + la_n - 2

In [ ]:
dof

In [ ]:
1 - scipy.stats.t.cdf(4.114, df=dof)

## with scipy

### Assuming equal variance

In [ ]:
scipy.stats.ttest_ind(oc.median_home_value, la.median_home_value, nan_policy="omit")

In [ ]:
scipy.stats.ttest_ind(
    oc.median_home_value, la.median_home_value, alternative="greater", nan_policy="omit"
)

### Assuming unequal variance (Welch's)

In [ ]:
scipy.stats.ttest_ind(
    oc.median_home_value.dropna(), la.median_home_value.dropna(), equal_var=False
)

## Excercise

np.float64(2107.520134228188)

np.float64(1654.647520661157)

I am a regional planner considering implementing a rent-control policy in southern california. I'm concerned that OC may have a higher rent burden than LA county. How can I test this hypothesis?

In [15]:
oc.median_contract_rent.mean()

np.float64(2107.520134228188)

In [16]:
la.median_contract_rent.mean()

np.float64(1654.647520661157)

null: OC <= LA rents
alt: OC > LA rents

In [18]:
scipy.stats.ttest_ind(oc.median_contract_rent, la.median_contract_rent, nan_policy='omit', equal_var=False, alternative='greater')

TtestResult(statistic=np.float64(18.232133294081756), pvalue=np.float64(5.7977341681919506e-64), df=np.float64(922.2656123633881))